In [ ]:
import numpy as np

In [ ]:
file = open("urdu.txt",'r')
lines = []
text = file.readlines()

In [ ]:
for line in text:
  lines.append(line)

In [ ]:
print(lines[0].split("\t")[1])

مدد۔


In [ ]:
encoder_input_data = None
decoder_input_data = None
decoder_target_data = None

In [ ]:
english = []
urdu = []

In [ ]:
for line in lines:
  english.append(line.split("\t")[0])
  urdu.append(line.split("\t")[1] + "\n")

In [ ]:
charEng = []
charUrdu = []


In [ ]:
for word in english:
  for ch in word: 
    if ch not in charEng:
      charEng.append(ch)

In [ ]:
for word in urdu:
  for ch in word:
    if ch not in charUrdu:
      charUrdu.append(ch)

In [ ]:
charEng = sorted(charEng)
charUrdu = sorted(charUrdu)

In [ ]:
max_pairs_english = len(english)
max_pairs_urdu = len(urdu)

max = ""
for sentence in english:
  if len(sentence) > len(max):
    max = sentence

max_english_sentence_length = len(max)

max = ""
for sentence in urdu:
  if len(sentence) > len(max):
    max = sentence

max_urdu_sentence_length = len(max)

num_english_characters = len(charEng)
num_urdu_characters = len(charUrdu)

In [ ]:
print("Max pairs English: ", max_pairs_english)
print("Max english sentence length: ", max_english_sentence_length)
print("Num of english char: ", num_english_characters)
print("Max pairs Urdu: ", max_pairs_urdu)
print("Max urdu sentence length: ", max_urdu_sentence_length)
print("Num of urdu char: ", num_urdu_characters)

Max pairs English:  1167
Max english sentence length:  94
Num of english char:  69
Max pairs Urdu:  1167
Max urdu sentence length:  87
Num of urdu char:  67


In [ ]:
charUrdu


['\n',
 ' ',
 '!',
 '"',
 '(',
 ')',
 '-',
 '.',
 '،',
 '؟',
 'آ',
 'ؤ',
 'ئ',
 'ا',
 'ب',
 'ت',
 'ث',
 'ج',
 'ح',
 'خ',
 'د',
 'ذ',
 'ر',
 'ز',
 'س',
 'ش',
 'ص',
 'ض',
 'ط',
 'ظ',
 'ع',
 'غ',
 'ف',
 'ق',
 'ل',
 'م',
 'ن',
 'و',
 'ي',
 'ً',
 'ُ',
 'ِ',
 'ّ',
 'ٓ',
 '٘',
 'ٹ',
 'پ',
 'چ',
 'ڈ',
 'ڑ',
 'ژ',
 'ک',
 'گ',
 'ں',
 'ھ',
 'ہ',
 'ی',
 'ے',
 '۔',
 '۰',
 '۱',
 '۲',
 '۳',
 '۵',
 '۷',
 '۸',
 '۹']

In [ ]:
batch_size = 64
epochs = 100
latent_dim = 256
train_length = 1000


In [ ]:
english_dict = {}
urdu_dict = {}

In [ ]:
for ch in charEng:
  english_dict[ch] = charEng.index(ch)
for ch in charUrdu:
  urdu_dict[ch] = charUrdu.index(ch)

In [ ]:
urdu_dict

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 '(': 4,
 ')': 5,
 '-': 6,
 '.': 7,
 '،': 8,
 '؟': 9,
 'آ': 10,
 'ؤ': 11,
 'ئ': 12,
 'ا': 13,
 'ب': 14,
 'ت': 15,
 'ث': 16,
 'ج': 17,
 'ح': 18,
 'خ': 19,
 'د': 20,
 'ذ': 21,
 'ر': 22,
 'ز': 23,
 'س': 24,
 'ش': 25,
 'ص': 26,
 'ض': 27,
 'ط': 28,
 'ظ': 29,
 'ع': 30,
 'غ': 31,
 'ف': 32,
 'ق': 33,
 'ل': 34,
 'م': 35,
 'ن': 36,
 'و': 37,
 'ي': 38,
 'ً': 39,
 'ُ': 40,
 'ِ': 41,
 'ّ': 42,
 'ٓ': 43,
 '٘': 44,
 'ٹ': 45,
 'پ': 46,
 'چ': 47,
 'ڈ': 48,
 'ڑ': 49,
 'ژ': 50,
 'ک': 51,
 'گ': 52,
 'ں': 53,
 'ھ': 54,
 'ہ': 55,
 'ی': 56,
 'ے': 57,
 '۔': 58,
 '۰': 59,
 '۱': 60,
 '۲': 61,
 '۳': 62,
 '۵': 63,
 '۷': 64,
 '۸': 65,
 '۹': 66}

In [ ]:
encoder_input_data = np.zeros((max_pairs_english, max_english_sentence_length,
                              num_english_characters), dtype='float32')
decoder_input_data = np.zeros((max_pairs_urdu, max_urdu_sentence_length, 
                               num_urdu_characters), dtype='float32')
decoder_target_data = np.zeros((max_pairs_urdu, max_urdu_sentence_length, 
                               num_urdu_characters), dtype='float32')

In [ ]:
for i, (input_text, target_text) in enumerate(zip(english, urdu)):
  for t, char in enumerate(input_text):
    encoder_input_data[i, t, english_dict[char]] = 1
  encoder_input_data[i, t + 1:, english_dict[' ']] = 1
  for t, char in enumerate(target_text):
    decoder_input_data[i, t, urdu_dict[char]] = 1
    if t > 0: 
      decoder_target_data[i, t - 1, urdu_dict[char]] = 1
  decoder_input_data[i, t + 1:, urdu_dict[' ']] = 1
  decoder_target_data[i, t:, urdu_dict[' ']] = 1

In [ ]:
encoder_input_data[0].shape

(94, 69)

In [ ]:
import keras as ks
from keras import models, layers


In [ ]:
encoder_inputs = layers.Input(shape=(None, num_english_characters))
encoder = layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

In [ ]:
decoder_inputs = layers.Input(shape=(None, num_urdu_characters))
decoder_lstm = layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = layers.Dense(num_urdu_characters, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
model = models.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit([encoder_input_data[:1000], decoder_input_data[:1000]], decoder_target_data[:1000], batch_size=batch_size, epochs=500, validation_split=0.2)

Epoch 1/500
13/13 [==============================] - 23s 106ms/step - loss: 2.4435 - accuracy: 0.5860 - val_loss: 1.5177 - val_accuracy: 0.6526
Epoch 2/500
13/13 [==============================] - 0s 24ms/step - loss: 1.0506 - accuracy: 0.7610 - val_loss: 1.4958 - val_accuracy: 0.6552
Epoch 3/500
13/13 [==============================] - 0s 22ms/step - loss: 1.0066 - accuracy: 0.7647 - val_loss: 1.5954 - val_accuracy: 0.6552
Epoch 4/500
13/13 [==============================] - 0s 22ms/step - loss: 1.0031 - accuracy: 0.7644 - val_loss: 1.5373 - val_accuracy: 0.6552
Epoch 5/500
13/13 [==============================] - 0s 23ms/step - loss: 0.9834 - accuracy: 0.7639 - val_loss: 1.4123 - val_accuracy: 0.6552
Epoch 6/500
13/13 [==============================] - 0s 24ms/step - loss: 0.9834 - accuracy: 0.7638 - val_loss: 1.3947 - val_accuracy: 0.6552
Epoch 7/500
13/13 [==============================] - 0s 22ms/step - loss: 0.9702 - accuracy: 0.7651 - val_loss: 1.3980 - val_accuracy: 0.6552
Epoc

In [ ]:
predictions = model.predict([encoder_input_data[1000:], decoder_input_data[1000:]])

In [ ]:
predictions[0][0]

array([3.7097602e-06, 6.7261010e-07, 7.3581070e-08, 1.4051358e-07,
       2.1667047e-06, 2.8751545e-06, 8.3967734e-08, 9.9680722e-07,
       6.5214419e-07, 1.0396527e-08, 4.5059105e-06, 4.0423394e-08,
       2.9198900e-05, 6.4289797e-04, 2.3287730e-04, 5.7517871e-04,
       6.6333126e-07, 4.3114227e-01, 6.6580495e-04, 1.7275261e-03,
       2.9308119e-04, 2.7255860e-06, 1.3108322e-03, 5.6980093e-06,
       3.6746040e-03, 3.1840555e-05, 1.0565494e-03, 7.6095131e-07,
       1.3422395e-05, 1.8401155e-05, 3.2992128e-05, 1.2467680e-05,
       9.9301484e-05, 7.2263110e-06, 4.5718689e-04, 1.4240181e-05,
       3.8074396e-04, 8.9135394e-03, 3.6074263e-05, 3.6131036e-07,
       1.2108586e-06, 7.9131127e-08, 9.1057480e-08, 3.8749639e-07,
       3.4927081e-07, 9.3991066e-06, 8.9077430e-06, 1.8632770e-04,
       1.6592909e-06, 5.8812839e-06, 1.0663297e-07, 1.2398465e-04,
       4.6878795e-05, 3.4242858e-06, 3.2640098e-06, 4.6617634e-04,
       5.4773742e-01, 1.6032399e-08, 2.3288472e-07, 5.8912306e